In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random

# --- CONFIGURATION ---
GRID_DIR = 'data/grids'
STATIC_PATH = os.path.join(GRID_DIR, 'static_terrain.npy')
DYNAMIC_DIR = os.path.join(GRID_DIR, 'dynamic')
TARGET_DIR = os.path.join(GRID_DIR, 'targets')

# --- 1. CHECK STATIC TERRAIN ---
if not os.path.exists(STATIC_PATH):
    print(f"❌ CRITICAL: Static terrain file not found at {STATIC_PATH}")
else:
    static_data = np.load(STATIC_PATH)
    print(f"✅ Static Terrain Loaded. Shape: {static_data.shape}")
    print(f"   Channels: Elevation, Slope, Aspect(Sin), Aspect(Cos)")

# --- 2. CHECK FILE COUNTS ---
dynamic_files = sorted([f for f in os.listdir(DYNAMIC_DIR) if f.endswith('.npz')])
target_files = sorted([f for f in os.listdir(TARGET_DIR) if f.endswith('.npy')])

print(f"✅ Found {len(dynamic_files)} daily weather grids.")
print(f"✅ Found {len(target_files)} daily target maps.")

if len(dynamic_files) == 0 or len(target_files) == 0:
    raise ValueError("❌ Missing dynamic or target files! Did the scripts run?")

# --- 3. VISUAL ALIGNMENT CHECK (THE MOST IMPORTANT PART) ---
# Pick a random date that exists in both sets
common_dates = list(set([f.split('.')[0] for f in dynamic_files]) & 
                    set([f.split('.')[0] for f in target_files]))
test_date = random.choice(common_dates)
print(f"\n🔎 Inspecting Data for Date: {test_date}")

# Load Data
weather_data = np.load(os.path.join(DYNAMIC_DIR, f"{test_date}.npz"))['data']
target_data = np.load(os.path.join(TARGET_DIR, f"{test_date}.npy"))

# Plotting
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Static Elevation (Background)
# We use origin='lower' because we confirmed your arrays are South-Up (Index 0 = South)
axs[0].imshow(static_data[:, :, 0], cmap='terrain', origin='lower')
axs[0].set_title("Static: Elevation (Normalized)")
axs[0].axis('off')

# Plot 2: Dynamic Feature (e.g., Channel 0 = HN72_24 / 3-Day Snow)
# Check your DYNAMIC_FEATURES list to know which index is which. Usually 0 is snow.
# Use a masked array to hide 0s if you want to see the shape better
axs[1].imshow(weather_data[:, :, 0], cmap='Blues', origin='lower') 
axs[1].set_title(f"Dynamic: Weather Channel 0 ({test_date})")
axs[1].axis('off')

# Plot 3: Target Labels (Danger Level)
# Mask out the -1 values (No Data / Borders) so they don't mess up the colors
masked_target = np.ma.masked_where(target_data < 0, target_data)
axs[2].imshow(masked_target, cmap='RdYlGn_r', origin='lower', vmin=1, vmax=4)
axs[2].set_title(f"Target: Danger Level ({test_date})")
axs[2].axis('off')

plt.show()